In [1]:
import os
os.chdir('../')
current_path = os.getcwd()
print(current_path)

C:\Users\Pascal Gitz\OneDrive - Hochschule Luzern\Dokumente\sympycalcs


In [1]:
import sympycalcs.helpers as sh
from sympycalcs import render, convert
import sympy as sp 

import sympy.physics.units as unit
from sympy.abc import *
sp.init_printing(use_latex='mathjax', latex_mode='equation')

# Beispiel: Kragarm mit 2 Punktmassen
{{< pagebreak >}}
## Aufgabenstellung

Das in @fig-kragarm_2_punkte dargestellte System stellt einen Kragarm mit verteilter Masse und 2 Punktmassen dar. Eine mögliche Formfunktion ist rechts daneben gezeigt.

![Kragarm mit verteilter Masse und zwei Punktmassen](bilder/aufgabe_rayleigh_2_massen.svg){#fig-kragarm_2_punkte}

Gesucht:

- Grundfrequenz (1. Eigenfrequenz $\omega_n$) des Systems, berechnet mit dem Rayleigh-Quotienten.

Gegeben:

- Randbedingungen für den Spezialfall: $m_{const} = 0 \text{ und } m_1 = m_2 = m$
- Formfunktion:
$$ \Psi(x) = 1 - \cos(\frac{\pi x}{2L})$$


In [3]:
L = sp.symbols('L', positive=True)
m_1, m_2, m_star, k_star, omega_1 = sp.symbols('m_1, m_2, m_star, k_star omega_1')

Psi_x = sp.Function('Psi')(x)
f = sp.Function('f')(x,t)

In [4]:
params={
    m:0,
    m_1:m,
    m_2:m
}


{{< pagebreak >}}
## Musterlösung

### Formfunktion

In [5]:
eq_Psi_x = Eqn(Psi_x, 1 - sp.cos(sp.pi*x/(2*L)))
eq_Psi_x

Equation(Psi(x), 1 - cos(pi*x/(2*L)))

### Allgemeine Bewegungsgleichung

Mithilfe der in der Vorlesung hergeleiteten Bewegungsgleichung kann anhand der Formfunktion $\Psi$ die erste Eigenkreisfrequenz ermittelt werden. Der Rayleigh-Quotient ist eine Energiebetrachtung. Er setzt die potentielle, maximale Energie $E_{pot,max}$ zur kinetischen, maximalen Energie $E_{kin,max}$ ins Verhältnis. Daraus lässt sich die Kreisfrequenz $\omega_n$ herauslösen.


In [6]:
eq_m_star = Eqn(m_star,sp.Integral(m*Psi_x**2,(x, 0,L))+m_1 * Psi_x.subs(x,L)**2 + m_2 *Psi_x.subs(x,L/2)**2)
eq_k_star = Eqn(k_star,sp.Integral(E*I*sp.Derivative(Psi_x,x,2)**2, (x,0,L)))
display(eq_m_star, eq_k_star)

Equation(m_star, m_1*Psi(L)**2 + m_2*Psi(L/2)**2 + Integral(m*Psi(x)**2, (x, 0
, L)))

Equation(k_star, Integral(E*I*Derivative(Psi(x), (x, 2))**2, (x, 0, L)))

In [7]:
eq_bewegung_allg = Eqn(f, sp.Derivative(u,x,2)*m_star+u*k_star)
eq_bewegung_allg

Equation(f(x, t), k_star*u + m_star*Derivative(u, (x, 2)))

In [8]:
eq_bewegung_allg.subs(eq_m_star, eq_k_star)

Equation(f(x, t), u*Integral(E*I*Derivative(Psi(x), (x, 2))**2, (x, 0, L)) + (
m_1*Psi(L)**2 + m_2*Psi(L/2)**2 + Integral(m*Psi(x)**2, (x, 0, L)))*Derivative
(u, (x, 2)))

Substituiert mit Massen- und Steifigkeitsvariable:

### Eigenkreisfrequenz

Aus der Bewegungsgleichung kann die Eigenkreisfrequenz ermittelt werden:

In [9]:
eq_eigenkreisfreq = Eqn(omega_1, sp.sqrt(k_star/ m_star))
eq_eigenkreisfreq

Equation(omega_1, sqrt(k_star/m_star))

#### Bestimmung der Masse

Die Masse kann mittels der Lösung des Integrals bestimmt werden. Dabei sind die Punktmassen mittels der entsprechenden Deformation an den Stellen $L$ und $\frac{L}{2}$ zu berücksichtigen, sowie die verteilte Masse über die gesamte Länge.


In [10]:
eq_Psi_x.subs(x,L)

Equation(Psi(L), 1)

In [11]:
eq_Psi_x.subs(x,L/2)

Equation(Psi(L/2), 1 - sqrt(2)/2)

In [12]:
eq_Psi_x**2

Equation(Psi(x)**2, (1 - cos(pi*x/(2*L)))**2)

In [13]:
eq_m_star.subs(eq_Psi_x, eq_Psi_x.subs(x,L), eq_Psi_x.subs(x,L/2))

Equation(m_star, m_1 + m_2*(1 - sqrt(2)/2)**2 + Integral(m*(1 - cos(pi*x/(2*L)
))**2, (x, 0, L)))

In [14]:
eq_m_star.subs(eq_Psi_x, eq_Psi_x.subs(x,L), eq_Psi_x.subs(x,L/2)).doit()

Equation(m_star, m*(-4*L/pi + 3*L/2) + m_1 + m_2*(1 - sqrt(2)/2)**2)

#### Bestimmung der Steifigkeit

Die Steifigkeit in kann mittels der Lösung des Integrals in bestimmt werden. 
Zur Ermittlung der Steifigkeit $k^\star$ muss zuerst der Ansatz zweimal nach $x$ abgeleitet werden.

In [15]:
display(eq_Psi_x,sp.diff(eq_Psi_x,x,1),sp.diff(eq_Psi_x,x,2))

Equation(Psi(x), 1 - cos(pi*x/(2*L)))

Equation(Derivative(Psi(x), x), pi*sin(pi*x/(2*L))/(2*L))

Equation(Derivative(Psi(x), (x, 2)), pi**2*cos(pi*x/(2*L))/(4*L**2))

Danach bedingt es lediglich das einsetzen:

In [16]:
display(eq_k_star.subs(eq_Psi_x),
       eq_k_star.subs(eq_Psi_x).doit())

Equation(k_star, Integral(E*I*Derivative(1 - cos(pi*x/(2*L)), (x, 2))**2, (x, 
0, L)))

Equation(k_star, pi**4*E*I/(32*L**3))

#### Grundfrequenz
Letztlich kann die Grundfrequenz bestimmt werden:

In [17]:
omega_1_sub = eq_eigenkreisfreq.subs(eq_k_star, eq_m_star).subs(eq_Psi_x, eq_Psi_x.subs(x,L), eq_Psi_x.subs(x,L/2)).simplify()

omega_1_sub

Equation(omega_1, sqrt(2)*pi**2*sqrt(E*I/(4*m*(-4*L/pi + 3*L/2) + 4*m_1 + m_2*
(-2 + sqrt(2))**2))/(4*L**(3/2)))

#### Auswertung des Spezialfalls

Mit Hilfe der Randbedingungen für den Spezialfall aus der Aufgabenstellung resultiert die Grundfrequenz zu:

In [18]:
omega_1_sub.subs(params).simplify().evalf(3)

Equation(omega_1, 1.67*(E*I/m)**0.5/L**1.5)